In [ ]:
 def rgb_to_xyz(self, rgb):
        """
        RGB转XYZ色彩空间
        使用sRGB标准转换矩阵
        """
        # 归一化RGB值到[0,1]
        rgb_norm = rgb / 255.0
        
        # Gamma校正
        rgb_linear = np.where(rgb_norm <= 0.04045,
                             rgb_norm / 12.92,
                             np.power((rgb_norm + 0.055) / 1.055, 2.4))
        
        # sRGB到XYZ的转换矩阵
        transform_matrix = np.array([
            [0.4124564, 0.3575761, 0.1804375],
            [0.2126729, 0.7151522, 0.0721750],
            [0.0193339, 0.1191920, 0.9503041]
        ])
        
        xyz = np.dot(rgb_linear, transform_matrix.T)
        return xyz


# sRGB->CIE XYZ空间的转换
$$
RGB = 
\begin{bmatrix}
  R\\
  G\\
  B\\
\end{bmatrix}
$$

$$RGB_{norm}=\frac{1}{255}RGB$$

$$
f(x) =
\begin{cases}
  \dfrac{x}{12.92}, & x \leq 0.04045 \\
  \left( \dfrac{x + 0.055}{1.055} \right)^{2.4}, & x > 0.04045
\end{cases}
$$

$$
RGB_{linear} = f(RGB_{norm})
$$

$$
Transforming\ Matrix = 
\left[
\begin{matrix}
  0.4124564 & 0.3575761 & 0.1804375\\
  0.2126729 & 0.7151522 & 0.0721750\\
  0.0193339 & 0.1191920 & 0.9503041\\
\end{matrix}
\right]
$$

$$
\begin{bmatrix}
  X\\
  Y\\
  Z\\
\end{bmatrix}
 = Transforming\  Matrix\cdot RGB_{linear}
$$

In [1]:
    def xyz_to_lab(self, xyz):
        """
        XYZ转CIE Lab色彩空间
        """
        # D65白点
        Xn, Yn, Zn = 0.95047, 1.00000, 1.08883
        
        x = xyz[..., 0] / Xn
        y = xyz[..., 1] / Yn
        z = xyz[..., 2] / Zn
        
        # 立方根变换
        fx = np.where(x > 0.008856, np.power(x, 1/3), (7.787 * x + 16/116))
        fy = np.where(y > 0.008856, np.power(y, 1/3), (7.787 * y + 16/116))
        fz = np.where(z > 0.008856, np.power(z, 1/3), (7.787 * z + 16/116))
        
        L = 116 * fy - 16
        a = 500 * (fx - fy)
        b = 200 * (fy - fz)
        
        return np.stack([L, a, b], axis=-1)

# CIE XYZ → CIE Lab 色彩空间转换

假设有颜色三刺激值向量：

$$
\begin{bmatrix}
X \\
Y \\
Z \\
\end{bmatrix}
$$

参考白点（D65）为：

$$
X_n = 0.95047,\quad Y_n = 1.00000,\quad Z_n = 1.08883
$$

归一化处理：

$$
x = \frac{X}{X_n},\quad y = \frac{Y}{Y_n},\quad z = \frac{Z}{Z_n}
$$

定义函数 \( f(t) \) 为：

$$
f(t) =
\begin{cases}
t^{1/3}, & t > 0.008856 \\
7.787t + \frac{16}{116}, & t \leq 0.008856
\end{cases}
$$

应用变换：

$$
f_x = f(x),\quad f_y = f(y),\quad f_z = f(z)
$$

最终得到 Lab 空间的三分量：

$$
\begin{aligned}
L^* &= 116 \cdot f_y - 16 \\
a^* &= 500 \cdot (f_x - f_y) \\
b^* &= 200 \cdot (f_y - f_z)
\end{aligned}
$$

组合成向量表示：

$$
Lab =
\begin{bmatrix}
L^* \\
a^* \\
b^* \\
\end{bmatrix}
$$

In [ ]:
    def calculate_color_difference(self, lab1, lab2, kL=1, kC=1, kH=1):
        L1, a1, b1 = lab1[..., 0], lab1[..., 1], lab1[..., 2]
        L2, a2, b2 = lab2[..., 0], lab2[..., 1], lab2[..., 2]

        # Step 1: C'
        C1 = np.sqrt(a1**2 + b1**2)
        C2 = np.sqrt(a2**2 + b2**2)
        C_bar = 0.5 * (C1 + C2)

        # Step 2: a' 修正
        G = 0.5 * (1 - np.sqrt(C_bar**7 / (C_bar**7 + 25**7)))
        a1p = (1 + G) * a1
        a2p = (1 + G) * a2

        C1p = np.sqrt(a1p**2 + b1**2)
        C2p = np.sqrt(a2p**2 + b2**2)

        # Step 3: h' 色相角
        h1p = np.degrees(np.arctan2(b1, a1p)) % 360
        h2p = np.degrees(np.arctan2(b2, a2p)) % 360

        # Step 4: Delta L', C', H'
        dLp = L2 - L1
        dCp = C2p - C1p

        dhp = h2p - h1p
        dhp = dhp - 360 * (dhp > 180) + 360 * (dhp < -180)
        dHp = 2 * np.sqrt(C1p * C2p) * np.sin(np.radians(dhp / 2))

        # Step 5: 加权系数
        L_bar = 0.5 * (L1 + L2)
        C_bar_p = 0.5 * (C1p + C2p)

        h_bar_p = (h1p + h2p + 360 * (np.abs(h1p - h2p) > 180)) / 2
        h_bar_p %= 360

        T = (1
            - 0.17 * np.cos(np.radians(h_bar_p - 30))
            + 0.24 * np.cos(np.radians(2 * h_bar_p))
            + 0.32 * np.cos(np.radians(3 * h_bar_p + 6))
            - 0.20 * np.cos(np.radians(4 * h_bar_p - 63)))

        Sl = 1 + (0.015 * (L_bar - 50)**2) / np.sqrt(20 + (L_bar - 50)**2)
        Sc = 1 + 0.045 * C_bar_p
        Sh = 1 + 0.015 * C_bar_p * T

        delta_theta = 30 * np.exp(-((h_bar_p - 275)/25)**2)
        Rc = 2 * np.sqrt(C_bar_p**7 / (C_bar_p**7 + 25**7))
        Rt = -np.sin(np.radians(2 * delta_theta)) * Rc

        # Step 6: Final formula
        dE = np.sqrt(
            (dLp / (kL * Sl))**2 +
            (dCp / (kC * Sc))**2 +
            (dHp / (kH * Sh))**2 +
            Rt * (dCp / (kC * Sc)) * (dHp / (kH * Sh))
        )

        return dE

### CIEDE2000 色差计算公式（ΔE\_{00}）

设两颜色在 CIE Lab 空间中分别为：

$$
Lab_1 = (L_1, a_1, b_1), \quad Lab_2 = (L_2, a_2, b_2)
$$

---

#### Step 1：计算色度

$$
C_1 = \sqrt{a_1^2 + b_1^2}, \quad C_2 = \sqrt{a_2^2 + b_2^2}
$$

$$
\bar{C} = \frac{C_1 + C_2}{2}
$$

---

#### Step 2：a 分量修正

$$
G = \frac{1}{2} \left(1 - \sqrt{\frac{\bar{C}^7}{\bar{C}^7 + 25^7}} \right)
$$

$$
a_1' = (1 + G) a_1, \quad a_2' = (1 + G) a_2
$$

$$
C_1' = \sqrt{a_1'^2 + b_1^2}, \quad C_2' = \sqrt{a_2'^2 + b_2^2}
$$

---

#### Step 3：色相角计算

$$
h_1' = \text{atan2}(b_1, a_1') \mod 360^\circ, \quad
h_2' = \text{atan2}(b_2, a_2') \mod 360^\circ
$$

---

#### Step 4：计算各分量差异

$$
\Delta L' = L_2 - L_1, \quad
\Delta C' = C_2' - C_1'
$$

$$
\Delta h' = h_2' - h_1'
$$

调整为最小角度差：

$$
\Delta h' =
\begin{cases}
\Delta h' - 360^\circ, & \Delta h' > 180^\circ \\
\Delta h' + 360^\circ, & \Delta h' < -180^\circ \\
\Delta h', & \text{otherwise}
\end{cases}
$$

$$
\Delta H' = 2 \sqrt{C_1' C_2'} \cdot \sin\left( \frac{\Delta h'}{2} \cdot \frac{\pi}{180} \right)
$$

---

#### Step 5：平均值与加权因子

亮度、色度、色相平均值：

$$
\bar{L} = \frac{L_1 + L_2}{2}, \quad
\bar{C}' = \frac{C_1' + C_2'}{2}
$$

$$
\bar{h}' =
\begin{cases}
\frac{h_1' + h_2'}{2} + 180^\circ, & |h_1' - h_2'| > 180^\circ \\
\frac{h_1' + h_2'}{2}, & \text{otherwise}
\end{cases}
$$

辅助项 \( T \)：

$$
T = 1
- 0.17 \cos(h' - 30^\circ)
+ 0.24 \cos(2 h')
+ 0.32 \cos(3 h' + 6^\circ)
- 0.20 \cos(4 h' - 63^\circ)
$$

加权函数：

$$
S_L = 1 + \frac{0.015 ( \bar{L} - 50 )^2}{\sqrt{20 + (\bar{L} - 50)^2}}, \quad
S_C = 1 + 0.045 \bar{C}', \quad
S_H = 1 + 0.015 \bar{C}' T
$$

旋转项：

$$
\Delta \theta = 30 \cdot \exp\left( -\left( \frac{\bar{h}' - 275}{25} \right)^2 \right)
$$

$$
R_C = 2 \sqrt{ \frac{\bar{C}'^7}{\bar{C}'^7 + 25^7} }, \quad
R_T = -R_C \cdot \sin\left( 2 \Delta \theta \cdot \frac{\pi}{180} \right)
$$

---

#### Step 6：最终色差公式（CIEDE2000）

$$
\Delta E_{00} = \sqrt{
\left( \frac{\Delta L'}{k_L S_L} \right)^2 +
\left( \frac{\Delta C'}{k_C S_C} \right)^2 +
\left( \frac{\Delta H'}{k_H S_H} \right)^2 +
R_T \cdot \left( \frac{\Delta C'}{k_C S_C} \right) \left( \frac{\Delta H'}{k_H S_H} \right)
}
$$

其中：

- $( k_L, k_C, k_H )$：为应用场景调整因子，通常取值均为 1；
- 所有角度（如色相角、Δθ）都以 **角度制** 处理，注意单位转换；
- $(\text{atan2}(b, a))$：为反正切函数，返回值在 $([-\pi, \pi])$，需转为 $([0, 360^\circ])$

